In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,BatchNormalization,Flatten
from tensorflow.keras.regularizers import l2

In [44]:
data_gen=ImageDataGenerator(rescale=(1.0/255),
                           rotation_range=20,
                           width_shift_range=0.2,
                           height_shift_range=0.2,
                           shear_range=0.2,
                           zoom_range=0.2,
                           horizontal_flip=True,fill_mode="nearest")

In [45]:
training_data=data_gen.flow_from_directory(directory="/kaggle/input/car-dataset/car_color_dataset/train",
                                          target_size=(244,244),batch_size=64,
                                          class_mode="categorical",save_format="jpg")
testing_data=data_gen.flow_from_directory(directory="/kaggle/input/car-dataset/car_color_dataset/train",
                                          target_size=(244,244),batch_size=120,
                                          class_mode="categorical",save_format="jpg")

Found 3802 images belonging to 11 classes.
Found 3802 images belonging to 11 classes.


In [34]:
print(len(training_data.labels))
print(len(training_data))
print(len(testing_data.labels))
print(len(testing_data))

3802
60
3802
32


In [14]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB7

In [46]:
vgg16 = VGG16(input_shape =[244,244,3],
              weights = "imagenet",
              include_top = False,
              classes=1000,
              classifier_activation='softmax')
eff_net=EfficientNetB7(
    include_top=False,
    weights='imagenet',
    input_shape=[244,244,3],
    classes=1000,
    classifier_activation='softmax',
)

In [47]:
vgg16.trainable=False

In [48]:
for layer in eff_net.layers[:-5]:
    layer.trainable = False

In [84]:
model1 = Sequential([
    vgg16,
    Flatten(),
    Dense(12, activation='relu'),
    Dropout(0.4),
    Dense(16, activation='relu'),
    Dropout(0.4),
    Dense(6, activation='relu'),
    Dropout(0.2),
    Dense(11, activation='softmax'),
])
model2= Sequential([
    eff_net,
    Flatten(),
    Dense(6, activation='relu'),
    Dense(12, activation='relu'),
    Dropout(0.4),
    Dense(6, activation='relu'),
    Dense(10, activation='softmax'),
])

In [89]:
from keras.callbacks import EarlyStopping
callback=EarlyStopping(monitor='val_loss',patience=20,mode='auto',baseline=None,restore_best_weights=True
                       ,start_from_epoch=10)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
model1.compile(optimizer="adam", loss= tf.keras.losses.CategoricalCrossentropy,metrics=["accuracy"])
history=model1.fit(x=training_data,validation_data=testing_data,epochs=50,callbacks=callback)

Epoch 1/50
20/60 ━━━━━━━━━━━━━━━━━━━━ 36s 909ms/step - accuracy: 0.1480 - loss: 2.3943